In [1]:
!pip install python-dotenv

In [2]:
!pip install psycopg2-binary

     |████████████████████████████████| 3.0MB 5.7MB/s 


In [3]:
import psycopg2
import pandas as pd
import json
import os
from dotenv import load_dotenv

# Convert CSV into DataFrame format

In [25]:
#Convert CSV File W/ Data into dataframe, make sure you have the correct file and in the proper csv format
df = pd.read_csv("predict_df.csv")
# df = pd.read_csv("main_data_clean.csv")

In [30]:
#Data frame looks good to go
print(df.shape)
df.head()

(1472, 7)


,bridge_opportunity_bridge_type,bridge_opportunity_span_m,days_per_year_river_is_flooded,bridge_classification,flag_for_rejection,height_differential_between_banks,bridge_opportunity_project_code
0,Suspended Bridge,52.44,78.63,NaN,Yes,0.97,1009317
1,Suspension Bridge,48.00,120.00,NaN,NaN,0.70,1007325
2,Suspension Bridge,50.00,60.00,Standard,NaN,1.00,1007374
3,Suspension Bridge,68.00,60.00,Standard,No,1.10,1007466
4,Suspension Bridge,78.00,35.00,Standard,No,1.10,1007495


# Clean Columns If Necessary

# Check Secrets

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [20]:
#Secrets are contained in a .envfile
load_dotenv()
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# print(db_name)
# print(db_user)
# print(db_password)
# print(db_host)
# print(db_port)

# Verify which tables exist in DataBase

In [31]:
import sqlalchemy  # Package for accessing SQL databases via Python
# Connect to database (Note: The package psychopg2 is required for Postgres to work with SQLAlchemy)
engine = sqlalchemy.create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
con = engine.connect()
# Verify that there are no existing tables
print(engine.table_names())
engine.dispose()

['b2p_lab28d_sk_tab']


# Upload DataFrame to DataBase

In [32]:
table_name = 'predict_sk_tab'
# table_name = 'b2p_lab28d_sk_tab'

df.to_sql(table_name, con)

In [ ]:
def conn_curs():
    """
    makes a connection to the database
    """
    global db_name
    global db_user
    global db_password
    global db_host
    global db_port
    
    connection = psycopg2.connect(dbname=db_name, user= db_user,
                                  password=db_password, host= db_host,port=db_port)
    cursor = connection.cursor()
    return connection, cursor

In [ ]:
def fetch_query_records(query):
    global conn_curs
    conn, cursor = conn_curs()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    return result

In [ ]:
fetch_query_records("""SELECT "bridge_name" from public."B2P_Merged_Final" LIMIT 1;""")

[('Bukinga',)]

In [ ]:
# Testing Querie to get Records based on Bridge Name
conn, cursor = conn_curs()
query  = f"""SELECT country from {table_name} where bridge_name = 'Bukinga' LIMIT 1;"""
cursor.execute(query)
result = cursor.fetchall()
cursor.close()
conn.close
print(result)

[('Rwanda',)]


In [ ]:
def fetch_all_records():
    global conn_curs
    conn,cursor = conn_curs()
    query = f""" SELECT * from public.{table_name} LIMIT 1""";
    cursor.execute(query)
    result = cursor.fetchall() #when it fetches it adds an extra column (indexes), make sure to remove it
    conn.close()
    df2 = pd.DataFrame(result, columns= ['index']+columns)
    df2 = df2.iloc[:,1:] # This code removes extra copied indexed column
    df_json = df.to_json(orient='records')
    parsed = json.loads(df_json)
    return df2


In [ ]:
#Testing fetch_all_records function
df2 = fetch_all_records()
print(df2.shape)
df2.head()

(1, 44)


,bridge_name,bridge_opportunity_project_code,bridge_opportunity_needs_assessment,bridge_opportunity_level1_government,bridge_opportunity_level2_government,bridge_opportunity_stage,bridge_opportunity_gps_latitude,bridge_opportunity_gps_longitude,bridge_opportunity_bridge_type,bridge_opportunity_span_m,bridge_opportunity_individuals_directly_served,bridge_opportunity_comments,form_form_name,form_created_by,proposed_bridge_location_gps_latitude,proposed_bridge_location_gps_longitude,current_crossing_method,nearest_all_weather_crossing_point,days_per_year_river_is_flooded,flood_duration_during_rainy_season,market_access_blocked_by_river,education_access_blocked_by_river,health_access_blocked_by_river,other_access_blocked_by_river,primary_occupations,primary_crops_grown,river_crossing_deaths_in_last_3_years,river_crossing_injuries_in_last_3_years,incident_descriptions,notes_on_social_information,cell_service_quality,four_wd _accessibility,name_of_nearest_city,name_of_nearest_paved_or_sealed_road,bridge_classification,flag_for_rejection,rejection_reason,bridge_type,estimated_span_m,height_differential_between_banks,bridge_opportunity_general_project_photos,bridge_opportunity_casesafeid,senior_engineering_review_conducted,country
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,Cancelled,-2.760833,29.488056,Suspended Bridge,None,None,Steep slopes to deal with for anchors,Project Assessment - 2018.10.10,aimablengirabakunzi taroworks,-2.760833,29.488056,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Huye,None,None,Yes,There is a planned vehicular road with a vehic...,None,None,None,None,006f100000asFErAAM,Yes,Rwanda


In [22]:
df.columns

Index(['bridge_name', 'bridge_opportunity_project_code',
       'bridge_opportunity_needs_assessment',
       'bridge_opportunity_level1_government',
       'bridge_opportunity_level2_government',
       'bridge_opportunity_gps_latitude', 'bridge_opportunity_gps_longitude',
       'bridge_opportunity_bridge_type', 'bridge_opportunity_span_m',
       'bridge_opportunity_individuals_directly_served',
       'bridge_opportunity_comments', 'form_form_name', 'form_created_by',
       'proposed_bridge_location_gps_latitude',
       'proposed_bridge_location_gps_longitude', 'current_crossing_method',
       'nearest_all_weather_crossing_point', 'days_per_year_river_is_flooded',
       'flood_duration_during_rainy_season', 'market_access_blocked_by_river',
       'education_access_blocked_by_river', 'health_access_blocked_by_river',
       'other_access_blocked_by_river', 'primary_occupations',
       'primary_crops_grown', 'river_crossing_deaths_in_last_3_years',
       'river_crossing_injuri